# Install JDK and Cassandra 
- Install JDK, set enviornment variable of JAVA_HOME Path
- Download Cassandra, set environment variable of CASSANDRA_HOME Path
- run CASSANDRA_HOME Path\bin\cassandra.bat to run the serves

In [39]:
from cassandra.cluster import Cluster
cluster=Cluster(['localhost'])
session=cluster.connect()

Traceback (most recent call last):
  File "cassandra\cluster.py", line 3519, in cassandra.cluster.ControlConnection._reconnect_internal
  File "cassandra\cluster.py", line 3541, in cassandra.cluster.ControlConnection._try_connect
  File "cassandra\cluster.py", line 1617, in cassandra.cluster.Cluster.connection_factory
  File "cassandra\connection.py", line 759, in cassandra.connection.Connection.factory
  File "C:\ProgramData\Anaconda3\lib\site-packages\cassandra\io\asyncorereactor.py", line 351, in __init__
    self._connect_socket()
  File "cassandra\connection.py", line 826, in cassandra.connection.Connection._connect_socket
ConnectionRefusedError: [Errno 10061] Tried connecting to [('::1', 9042, 0, 0)]. Last error: No connection could be made because the target machine actively refused it


In [40]:
session.execute("drop KEYSPACE stocks")

In [41]:
session.execute("CREATE KEYSPACE stocks \
                    WITH replication={'class':'SimpleStrategy',\
                    'replication_factor':1}")

In [43]:
session.set_keyspace('stocks')

In [44]:
session.execute("""
                    CREATE TABLE company(
                        company_id text,
                        name_latest text,
                        name_previous text,
                        PRIMARY KEY (company_id)
                        )
                        """)

In [45]:
session.execute("""
                    CREATE TABLE indicator_by_company(
                        company_id text,
                        indicator_id text,
                        yr_2010 bigint,
                        yr_2011 bigint,
                        yr_2012 bigint,
                        yr_2013 bigint,
                        yr_2014 bigint,
                        yr_2015 bigint,
                        yr_2016 bigint,
                        PRIMARY KEY (company_id,indicator_id)
                        )
                        """)

# Load data into Cassandra using Json
- import json module
- use json modul to open json file into list(dict)
- use session.excute to insert loaded files into cassandra

In [46]:
import json
with open('data\companies.json') as f_in:
    companies = json.load(f_in)
print("type of loaded json file:",type(companies))
companies[3]

type of loaded json file: <class 'list'>


{'company_id': '1000228',
 'name_latest': 'Henry Schein Inc',
 'names_previous': None}

In [47]:
print("elements in list:",type(companies[0]))

elements in list: <class 'dict'>


In [48]:
session.execute(
                """
                INSERT INTO company (company_id, name_latest, name_previous)
                VALUES (%s ,%s ,%s)
                """,
                ('1000228','Henry Schein Inc',''))

In [49]:
result_set = session.execute(""" SELECT * FROM COMPANY""")
result_set.current_rows

[Row(company_id='1000228', name_latest='Henry Schein Inc', name_previous='')]

- insert all rows in json file into cassandra

In [50]:
for company in companies:
    
    try:
        session.execute(
                """
                INSERT INTO company (company_id, name_latest, name_previous)
                VALUES (%s ,%s ,%s)
                """,
                (company['company_id'],company['name_latest'],company['names_previous']))
    except:
        pass
    

In [51]:
result_set = session.execute(""" SELECT * FROM COMPANY""")
result_set.current_rows

ice Hotels International Inc /DE', name_previous=None),
 Row(company_id='1448962', name_latest='YA ZHU Silk, Inc.', name_previous=None),
 Row(company_id='1282400', name_latest='Mewbourne Energy Partners 04-A LP', name_previous=None),
 Row(company_id='315261', name_latest='Texas Vanguard OIL CO', name_previous=None),
 Row(company_id='1349976', name_latest='American Standard Energy Corp.', name_previous=None),
 Row(company_id='1587246', name_latest='Cypress Energy Partners, L.P.', name_previous=None),
 Row(company_id='788885', name_latest='Pennichuck Corp', name_previous=None),
 Row(company_id='1404641', name_latest='Fidus Mezzanine Capital LP', name_previous=None),
 Row(company_id='885551', name_latest='Valence Technology Inc', name_previous=None),
 Row(company_id='804217', name_latest='Secured Income L P', name_previous=None),
 Row(company_id='1603145', name_latest='Nextera Energy Partners, LP', name_previous=None),
 Row(company_id='1423243', name_latest='Srkp 26 Inc', name_previous=No